In [1]:
import time 
import multiprocessing as mp
from multiprocessing import Pool
from collections import defaultdict
import sys
sys.path.append('/home/ak/Documents/PaperCode/stylised_facts')
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import lob_for_futures as lobFut
from lob_for_futures import *

from scipy.stats import jarque_bera
import matplotlib.pyplot as plt

from dateutil.parser import parse
from pandas.tseries.offsets import BDay
import pickle as pkl
import fnmatch
import pickle
import datetime as dt
from datetime import timedelta
import glob
import fathon
from fathon import fathonUtils as fu
import itertools
import pyinform as pyinf
# import pingouin as pig
import seaborn as sns
from scipy.stats import entropy
from scipy.spatial.distance import jensenshannon
from scipy.special import kl_div
import time
import pickle as pkl
import matplotlib.ticker as mtick

from functools import partial

In [2]:
loc_activityClockData = '/media/ak/My Passport/ActivityClockData'
loc_ProcessedDataForExperiments = '/media/ak/My Passport1/ExperimentData/ProcessedDataForExperiments'
list_activityClockData = os.listdir('/media/ak/My Passport/ActivityClockData')
list_ProcessedDataForExperiments = os.listdir('/media/ak/My Passport1/ExperimentData/ProcessedDataForExperiments')

In [ ]:
# !pip install pyinform

In [6]:


# Lets create a function that shifted your dataframe first before calling the corr().
# Let us break down what we hope to accomplish, and then translate that into code.
# For each hour (row) and for a given feature (column) I would like to find the value
# for that feature N hours prior.
# For each value of N (1-6 in our case) I want to make a new column
# for that feature representing the Nth prior hour's measurement.

elements = '/media/ak/Elements/OrderBookProcessedData'
laCie = '/media/ak/LaCie'
experimentsLocation = '/media/ak/T7/June4th2022Experiments'
bars =['volume_bar', 'calendar_bar', 'usd_volume_bar', 'tick_bar']
# plt.style.use(os.path.join('/home/ak/.config/matplotlib','l3.mplstyle'))
figures_destination = '/home/ak/Documents/Research/Papers/figures/'

jb_dict = dict()
standarised_returns = defaultdict(dict)
dataFolder = laCie
quotes = [f for f in os.listdir(dataFolder) if str('_quotes') in f]
trades = [f for f in os.listdir(dataFolder) if str('_trades') in f]

In [7]:
def print_cube(num):
    """
    function to print cube of given num
    """
    print("Cube: {}".format(num * num * num))


In [8]:
 
def print_square(num):
    """
    function to print square of given num
    """
    print("Square: {}".format(num * num))


In [10]:

# if __name__ == '__main__':
symbolIdx = 0
symbols = sorted(os.listdir(dataFolder))
procsdSymbolFolder = os.path.join(dataFolder, symbols[symbolIdx])
files = sorted(os.listdir(procsdSymbolFolder))
bars = ['volume_bar', 'calendar_bar', 'usd_volume_bar', 'tick_bar']
symbol = symbols[symbolIdx]
print(symbol)

#     pickle_out_filename = os.path.join( experimentsLocation, "".join((str(symbol), "_JB_results.pkl")))

#     pickle.dump(jb_dict, open(pickle_out_filename, 'wb'), protocol=pickle.HIGHEST_PROTOCOL)

#     pickle_out_returns = os.path.join(experimentsLocation, "".join((str(symbol), "_standard_returns.pkl")))
#     pickle.dump(standarised_returns, open(pickle_out_returns, 'wb'), protocol=pickle.HIGHEST_PROTOCOL)






.Trash-1000


In [ ]:


def jb_calculation(symbolIdx, filesIdx):

    # add location variables. these i have to figure how to abstract away
    print(symbols[symbolIdx])
    procsdSymbolFolder = os.path.join(elements, symbols[symbolIdx])
    print(procsdSymbolFolder)
    files = sorted(os.listdir(procsdSymbolFolder))
    
    fileLocation = os.path.join(procsdSymbolFolder, files[filesIdx])
    print(fileLocation)
    
    # pick the various files
    volume_bar_dict = open_pickle_filepath(fileLocation)[bars[0]]
    calendar_bar_dict = open_pickle_filepath(fileLocation)[bars[1]]
    usd_volume_bar_dict = open_pickle_filepath(fileLocation)[bars[2]]
    tick_bar_dict = open_pickle_filepath(fileLocation)[bars[3]]
    # get the dataframes

    volume_bar_df = volume_bar_dict[list(volume_bar_dict.keys())[0]]
    calendar_bar_df = calendar_bar_dict[list(calendar_bar_dict.keys())[0]]
    usd_volume_df = usd_volume_bar_dict[list(usd_volume_bar_dict.keys())[0]]
    tick_bar_df = tick_bar_dict[list(usd_volume_bar_dict.keys())[0]]
    # returns

    vb_ret = returns(volume_bar_df.micro_price_close).replace([np.inf, -np.inf], 0)  # volume
    tb_ret = returns(tick_bar_df.micro_price_close).replace([np.inf, -np.inf], 0)  # tick
    usdvb_ret = returns(usd_volume_df.micro_price_close).dropna().replace([np.inf, -np.inf], 0)  # usd volume
    cb_ret = returns(calendar_bar_df.micro_price_close).dropna().replace([np.inf, -np.inf], 0)  # calendar
    # calculating JB statistic
    jb_value_tick, _ = jarque_bera(tb_ret)
    jb_value_vol, _ = jarque_bera(vb_ret)
    jb_value_dollar, _ = jarque_bera(usdvb_ret)
    jb_value_calendar, _ = jarque_bera(cb_ret)


    jb_test_df = pd.DataFrame(data={'jarque_bera_results': [jb_value_tick,
                                                            jb_value_vol,
                                                            jb_value_dollar,
                                                            jb_value_calendar]
                                    },
                              index=['tick', 'vol', 'dollar', 'calendar'])
    pickle_out_returns = os.path.join(experimentsLocation, "".join((str(symbols[symbolIdx]), "_"+str(filesIdx)+"_jb_stats.pkl")))
    pickle.dump(jb_test_df, open(pickle_out_returns, 'wb'), protocol=pickle.HIGHEST_PROTOCOL)

    print("produced and saved JB stats for :", symbols[symbolIdx], filesIdx)


    

In [ ]:
jb_calculation(4, 4)
os.listdir('/media/ak/Elements/OrderBookProcessedData/')

In [ ]:
def standarised_returns(symbolIdx, filesIdx):
    standarised_returns = defaultdict(dict)
    # add location variables. these i have to figure how to abstract away
    ### all files here ###
    procsdSymbolFolder = os.path.join(elements, symbols[symbolIdx])
    files = sorted(os.listdir(procsdSymbolFolder))
    fileLocation = os.path.join(procsdSymbolFolder, files[filesIdx])
    
    volume_bar_dict = open_pickle_filepath(fileLocation)[bars[0]]
    calendar_bar_dict = open_pickle_filepath(fileLocation)[bars[1]]
    usd_volume_bar_dict = open_pickle_filepath(fileLocation)[bars[2]]
    tick_bar_dict = open_pickle_filepath(fileLocation)[bars[3]]
    ### all dataframes here ###

    volume_bar_df = volume_bar_dict[list(volume_bar_dict.keys())[0]]
    calendar_bar_df = calendar_bar_dict[list(calendar_bar_dict.keys())[0]]
    usd_volume_df = usd_volume_bar_dict[list(usd_volume_bar_dict.keys())[0]]
    tick_bar_df = tick_bar_dict[list(usd_volume_bar_dict.keys())[0]]


    vb_ret = returns(volume_bar_df.micro_price_close).replace([np.inf, -np.inf], 0)  # volume
    tb_ret = returns(tick_bar_df.micro_price_close).replace([np.inf, -np.inf], 0)  # tick
    usdvb_ret = returns(usd_volume_df.micro_price_close).dropna().replace([np.inf, -np.inf], 0)  # usd volume
    cb_ret = returns(calendar_bar_df.micro_price_close).dropna().replace([np.inf, -np.inf], 0)  # calendar
    
    tick_standard = (tb_ret - tb_ret.mean()) / tb_ret.std()
    volume_standard = (vb_ret - vb_ret.mean()) / vb_ret.std()
    dollar_standard = (usdvb_ret - usdvb_ret.mean()) / usdvb_ret.std()
    cb_standard = (cb_ret - cb_ret.mean()) / cb_ret.std()
    standarised_returns['tick'][filesIdx] = tick_standard
    standarised_returns['calendar'][filesIdx] = cb_standard
    standarised_returns['volume'][filesIdx] = volume_standard
    standarised_returns['dollar'][filesIdx] = dollar_standard
    
    pickle_out_returns = os.path.join(experimentsLocation, "".join((str(symbol), "_"+str(filesIdx)+"_standard_returns.pkl")))
    pickle.dump(standarised_returns, open(pickle_out_returns, 'wb'), protocol=pickle.HIGHEST_PROTOCOL)
    return standarised_returns




In [ ]:
test =standarised_returns(1, 1)

In [ ]:
test['tick']

In [ ]:
# if __name__ == "__main__":
#     # creating processes
#     p1 = multiprocessing.Process(target=jb_calculation, args=(1, ))
    
 
#     # starting process 1
#     p1.start()
#     # starting process 2
# #     p2.start()
 
#     # wait until process 1 is finished
# #     p1.join()
#     # wait until process 2 is finished
# #     p2.join()
 
#     # both processes finished
#     print("Done!")

In [ ]:
# from multiprocessing import Pool
# with Pool(5) as p:
#     print(p.map(jb_calculation, [1, 2, 3, 5, 6 ,7, 8, 25]))

In [ ]:
from functools import partial
from itertools import repeat
from multiprocessing import Pool, freeze_support

In [ ]:

def main():
    a_args = [1, 2]
    second_arg = [1, 2, 3, 4, 5, 6]
    with Pool() as pool:
#        L = pool.starmap(standarised_returns, [(1, 1), (1, 2), (1, 3)])
        K = pool.starmap(standarised_returns, list(itertools.product(a_args, second_arg)))
#         M = pool.starmap(jb_calculation, zip(a_args, second_arg))
#         N = pool.map(partial(jb_calculation, second_arg), a_args)
#         assert L == K

if __name__=="__main__":



In [ ]:
a_args = [1]
second_arg = [f for f in range(0, 20, 1)]

In [ ]:
# freeze_support()
# with Pool() as pool:
#     L = pool.starmap(standarised_returns, list(itertools.product(a_args, second_arg)))
#     K = pool.starmap(jb_calculation, list(itertools.product(a_args, second_arg)))

In [ ]:
print(L)

In [ ]:
list(zip(itertools.product(FIRST, SECOND)))

In [ ]:
test_dict =open_pickle_filepath('/media/ak/T7/June4th2022Experiments/DU1_standard_returns.pkl')

In [ ]:
test_dict.keys()

In [ ]:
files =[f for f in os.listdir('/media/ak/T7/June4th2022Experiments/') if str('_standard_returns') in f]

In [ ]:
symbol_ =' OE1'
type_file ='_standard_returns'
symbol_type_files = [f for f in files if str(symbol) in f if str(type_file) in f]
bars = ['tick', 'calendar', 'volume', 'dollar']
symbol_type_files

In [ ]:
list(oaded_returns['tick'].keys())[0]

In [ ]:
list(symbol_type_files

In [ ]:
file_index = 1
file_path =os.path.join(experimentsLocation, symbol_type_files[file_index])
loaded_returns = open_pickle_filepath(file_path)
index_item = list(loaded_returns[bars[0]].keys())[0]

In [ ]:
tr = loaded_returns['tick'][index_item]
calr = loaded_returns['calendar'][index_item]
vr = loaded_returns['volume'][index_item]
dr = loaded_returns['dollar'][index_item]




In [ ]:

tick_dict = dict()
usd_dict = dict()
calendar_dict = dict()
volm_dict = dict()
for file_index, _ in enumerate(symbol_type_files):
    file_path =os.path.join(experimentsLocation, symbol_type_files[file_index])
    loaded_returns = open_pickle_filepath(file_path)
    index_item = list(loaded_returns[bars[0]].keys())[0]
    tick_dict[file_index+1]= (pd.DataFrame(loaded_returns['tick'][index_item].values))
    usd_dict[file_index+1]= (pd.DataFrame(loaded_returns['dollar'][index_item].values))
    calendar_dict[file_index+1]= (pd.DataFrame(loaded_returns['calendar'][index_item].values))
    volm_dict[file_index+1]= (pd.DataFrame(loaded_returns['volume'][index_item].values))
    
    
    

In [ ]:
volm_dict

In [ ]:
user_dict = calendar_dict
def produce_dict(user_dict):
    columns_values = list(user_dict.keys())
    _df =pd.DataFrame.from_dict({(i,j): user_dict[i][j] 
                               for i in user_dict.keys() 
                               for j in user_dict[i].keys()},
                           orient='index')
    return_df = _df.reset_index().drop(columns=['level_0', 'level_1'])
    return return_df


In [ ]:
tick_ = tick_df.T.median(axis=1)
usd_ = usd_df.T.median(axis=1)
volm_ = volm_df.T.median(axis=1)
calendar_ = calendar_df.T.median(axis=1)


In [ ]:
tick_median = df.T.median(axis=1)

In [ ]:
import seaborn as sns
fig, ax = plt.subplots(figsize=(10,10))
sns.kdeplot(tick_, 
            ax=ax,
            label="Tick", 
            bw=0.25,
            color='darkblue')
sns.kdeplot(usd_,
            ax=ax,
            label="USD Volume",
            bw=1.55,linewidth=2.25,
            color='purple', linestyle ='-')

sns.kdeplot(volm_,
            ax=ax,
            label="Volume",
            bw=0.55,linewidth=1.25,
            color='red', linestyle ='-')
sns.kdeplot(calendar_,
            ax=ax,
            label="Calendar",
            bw=0.55,linewidth=2.25,
            color='grey', linestyle =':')

sns.kdeplot(np.random.normal(size=len(dr)),
            ax=ax,
            label="Normal",
            color='darkgreen',
            linewidth=3,
            linestyle="--")
plt.legend()


_ =plt.xlabel('Returns', fontsize=19)
_ =plt.ylabel('Density', fontsize=19)
_ =plt.xticks(fontsize = 15, rotation = 45)
_ = plt.yticks(fontsize = 15)

#plt.title(str(symbol))
file_name = str(symbol) + 'median_multiclocks_density_plot.png'
plt.savefig(os.path.join(figures_destination,file_name))
plt.legend()
plt.show()